In [15]:
#libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

import jax.random as jr
import jax.numpy as jnp
import jax
from dynamax.hidden_markov_model import GaussianHMM, DiagonalGaussianHMM, LinearAutoregressiveHMM

import os
import glob

In [16]:
#utils
def load_fmri(data_root, file_pattern):
    data = []
    subjects = []
    folders = sorted([
        f for f in os.listdir(data_root)
        if os.path.isdir(os.path.join(data_root, f))
    ])
    
    for folder in folders:
        matched_files = glob.glob(os.path.join(data_root, folder, file_pattern))
        if not matched_files:
            print(f"No files found matching {file_pattern} in {folder}")
            continue
            
        for file_path in matched_files:
            try:
                with h5py.File(file_path, 'r') as hdf:
                    data.append(hdf["dataset"][:])
                    subjects.append(folder)
            except Exception as e:
                print(f"Error loading {file_path}: {str(e)}")
                
    return data, subjects

In [17]:
# Load the data
data_root = "/home/zenari/data/fMRI_timeseries_clean_denoised_GSR_parcellated"
file_filter = "*task-hammer*.h5"
data, subjects = load_fmri(data_root, file_filter)

Error loading /home/zenari/data/fMRI_timeseries_clean_denoised_GSR_parcellated/NDAR_INVAG023WG3/task-hammerAP_run-01_bold_Atlas_hp2000_clean_GSR_parcellated.h5: name 'h5py' is not defined
Error loading /home/zenari/data/fMRI_timeseries_clean_denoised_GSR_parcellated/NDAR_INVAG339WHH/task-hammerAP_run-01_bold_Atlas_hp2000_clean_GSR_parcellated.h5: name 'h5py' is not defined
Error loading /home/zenari/data/fMRI_timeseries_clean_denoised_GSR_parcellated/NDAR_INVAG388HJL/task-hammerAP_run-01_bold_Atlas_hp2000_clean_GSR_parcellated.h5: name 'h5py' is not defined
Error loading /home/zenari/data/fMRI_timeseries_clean_denoised_GSR_parcellated/NDAR_INVAG900RVD/task-hammerAP_run-01_bold_Atlas_hp2000_clean_GSR_parcellated.h5: name 'h5py' is not defined
Error loading /home/zenari/data/fMRI_timeseries_clean_denoised_GSR_parcellated/NDAR_INVAH529JMM/task-hammerAP_run-01_bold_Atlas_hp2000_clean_GSR_parcellated.h5: name 'h5py' is not defined
Error loading /home/zenari/data/fMRI_timeseries_clean_denois

### HMM